In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d thedevastator/cancer-patients-and-air-pollution-a-new-link
!unzip \*.zip && rm *.zip

  0% 0.00/7.36k [00:00<?, ?B/s]
100% 7.36k/7.36k [00:00<00:00, 9.59MB/s]
Archive:  cancer-patients-and-air-pollution-a-new-link.zip
  inflating: cancer patient data sets.csv  


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load data
data = pd.read_csv('/content/cancer patient data sets.csv')

# Assuming 'Level' is the name of your target variable
X = data.drop(['index', 'Patient Id', 'Alcohol use', 'Dust Allergy', 'OccuPational Hazards', 'Genetic Risk', 'Level'], axis=1)
y = data['Level']

# Encode categorical variables (Gender)
label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=32)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Compute k nearest neighbors
k = 5  # Choose the value of k
nbrs = NearestNeighbors(n_neighbors=k+1).fit(X_train_scaled)  # k+1 because the nearest neighbor is the data point itself
distances, indices = nbrs.kneighbors(X_train_scaled)

# Determine mutual nearest neighbors and assign class label
y_pred = []
for i, neighbors in enumerate(indices):
    mutual_neighbors = list(set(neighbors).intersection(set(indices[neighbors[1]])))
    neighbor_labels = y_train.iloc[mutual_neighbors]  # Convert mutual_neighbors to list before indexing
    counter = Counter(neighbor_labels)
    majority_class = counter.most_common(1)[0][0]
    y_pred.append(majority_class)

# Calculate evaluation metrics
accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred, average='weighted')
recall = recall_score(y_train, y_pred, average='weighted')
f1 = f1_score(y_train, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
# Display classification report
print("\nClassification Report:\n", classification_report(y_train, y_pred))


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Classification Report:
               precision    recall  f1-score   support

        High       1.00      1.00      1.00       219
         Low       1.00      1.00      1.00       184
      Medium       1.00      1.00      1.00       197

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



In [4]:
# prompt: CREATE A CONFUSSION MATRIX

from sklearn.metrics import confusion_matrix

# Predict the labels for the test set
y_pred_test = []
for i, neighbors in enumerate(indices):
    mutual_neighbors = list(set(neighbors).intersection(set(indices[neighbors[1]])))
    neighbor_labels = y_train.iloc[mutual_neighbors]
    counter = Counter(neighbor_labels)
    majority_class = counter.most_common(1)[0][0]
    y_pred_test.append(majority_class)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# Print the confusion matrix
print("\nConfusion Matrix:\n", cm)


ValueError: Found input variables with inconsistent numbers of samples: [400, 600]